<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/ann-for-survey-sampling/blob/main/ann_paper_simulation_1_properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About

This notebook covers simulation study to assess the performance of predictive mean imputation based on exact and approximate nearest neigbours. 

**Warning**: before runing this scripts go to `Runtime` -> `Change runtime type` and set it to `GPU`.

## Install requested modules

Please note that this may take some time

In [1]:
!apt install libomp-dev
!pip install faiss-gpu
!pip install n2
!pip install scann
!pip install annoy ## takes minutes to add data and index
!pip install pyflann-py3
## !pip install pynndescent -- not suitable for PMM 1d dimension: gets error "no suitable hyperplains were found"

## this line cleanes information about installing
## comment these lines if you want to see the progress 
from IPython.display import clear_output 
clear_output()

## Import requested modules

In [3]:
## standard modules
import pandas as pd
import numpy as np
import time

## linear regression
from sklearn.linear_model import LinearRegression

## ann modules
import scann
import faiss
from pyflann import *
from annoy import AnnoyIndex
from n2 import HnswIndex
from scipy.spatial import cKDTree
from pynndescent import NNDescent

## serialization
import pickle

## Helper functions

In [11]:
def kdtree_impute(y_pred, y_pred_miss, y, eps = 0):
  tree = cKDTree(y_pred, leafsize = 100, balanced_tree=True)
  dists, indx = tree.query(y_pred_miss, k = 1, eps = eps)
  res = (np.sum(y) + np.sum(y[indx])) / (len(y_pred) + len(y_pred_miss))
  return res

def faiss_impute(y_pred, y_pred_miss, y, gpu = True, voronoi = False):
  index_flat = faiss.IndexFlatL2(1)

  if voronoi:
    index_flat = faiss.IndexIVFFlat(index_flat, 1, 1000)

  if gpu:
    gpu_faiss = faiss.StandardGpuResources() 
    index_flat = faiss.index_cpu_to_gpu(gpu_faiss, 0, index_flat)
  
  if voronoi:
    index_flat.train(y_pred)

  index_flat.add(y_pred)
  dists, indx = index_flat.search(y_pred_miss, k = 1) 
  res = (np.sum(y) + np.sum(y[indx])) / (len(y_pred) + len(y_pred_miss))
  return res

def annoy_impute(y_pred, y_pred_miss, y, trees = 50):
  t = AnnoyIndex(1, "euclidean") 
  for i in range(len(y_pred)):
    t.add_item(i, y_pred[i]) 

  t.build(trees)
  indx = np.array([t.get_nns_by_vector(i, 1) for i in y_pred_miss])
  res = (np.sum(y) + np.sum(y[indx])) / (len(y_pred) + len(y_pred_miss))
  return res

def n2_impute(y_pred, y_pred_miss, y, trees = 50):
  t = HnswIndex(1, "euclidean") 
  for i in y_pred:
    t.add_data(i)

  t.build(m=5, n_threads=-1)
  indx = t.batch_search_by_vectors(y_pred_miss, 1)
  res = (np.sum(y) + np.sum(y[indx])) / (len(y_pred) + len(y_pred_miss))
  return res

def flann_impute(y_pred, y_pred_miss, y, eps = 0):
  flann = FLANN()
  indx, dists = flann.nn(y_pred, y_pred_miss, 1, 
                         algorithm="kmeans", branching=32, iterations=7, 
                         checks=16, random_seed = 1, eps = eps)
  res = (np.sum(y) + np.sum(y[indx])) / (len(y_pred) + len(y_pred_miss))
  return res

def scann_impute(y_pred, y_pred_miss, y):
  searcher = scann.scann_ops_pybind.builder(y_pred, 1, "squared_l2").tree(
      num_leaves=1000, num_leaves_to_search=50, training_sample_size=5000).score_ah(
          2, anisotropic_quantization_threshold=0.2).reorder(10).build()
  indx, nns = searcher.search_batched(y_pred_miss)
  res = (np.sum(y) + np.sum(y[indx])) / (len(y_pred) + len(y_pred_miss))
  return res


## Simulation studies outline

Here, we conduct simulation study based on predictive mean matching. We replicate study from *Yang, S., & Kim, J. K. (2020). Asymptotic theory and inference of predictive mean matching imputation using a superpopulation model framework. Scandinavian Journal of Statistics, 47(3), 839-861.* paper, however we only assume missing data mechanism



In [ ]:
np.random.seed(123)
N = 50000
x1 = np.random.uniform(size = N)
x2 = np.random.uniform(size = N)
x3 = np.random.uniform(size = N)
x4 = np.random.normal(size = N)
x5 = np.random.normal(size = N)
x6 = np.random.normal(size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = -1 + x1 + x2 + epsilon
y2 = -1.167 + x1 + x2 + (x1 - 0.5)**2 + + (x2 - 0.5)**2 + epsilon
y3 = -1.5 + x1 + x2 + x3 + x4 + x5 + x6 + epsilon

## response indicator
p1 = np.exp(0.2 + x1 + x2) / (1 + np.exp(0.2 + x1 + x2))

data = np.column_stack((x1,x2,x3,x4,x5,x6,y1,y2,y3, p1)).astype('float32')

## first three rows
data[:3]

array([[ 0.6964692 ,  0.36086547,  0.20932843,  0.47665665,  0.80429375,
        -0.9894137 ,  0.42543107,  0.31638962,  0.42629617,  0.77856696],
       [ 0.28613934,  0.22535679,  0.2937351 ,  0.6701647 ,  1.2931948 ,
         1.033963  ,  0.25331086,  0.20747614,  3.0443685 ,  0.67073166],
       [ 0.22685145,  0.50813043,  0.05571789,  0.5033514 ,  1.5591047 ,
         0.13504769,  0.36729714,  0.27497336,  2.1205187 ,  0.71808493]],
      dtype=float32)

## Simulation study

Note, that this simulation may take several hours and you need to be connected to the Interent.

Results are stored in a `pickle` format that is naive for `python`'s `DataFrame`

In [ ]:
R = 500
sim1_results_ckdtree = np.zeros(shape = (R, 3))
sim1_results_faiss = np.zeros(shape = (R, 3))
sim1_results_faiss_v = np.zeros(shape = (R, 3))
sim1_results_annoy = np.zeros(shape = (R, 3))
sim1_results_n2 = np.zeros(shape = (R, 3))
sim1_results_flann = np.zeros(shape = (R, 3))
sim1_results_scann = np.zeros(shape = (R, 3))

for r in range(R):
  
  if (r % 10 == 0):
    print(r)

  np.random.seed(r)
  response_flag = np.random.binomial(n=1, p = p1, size = N)
  data_resp = data[response_flag == 1]
  data_noresp = data[response_flag != 1]
  
  ## predictive mean matching
  ## y1
  m1_reg_y1 = LinearRegression().fit(data_resp[:,:2], data_resp[:, 6])
  m1_resp_y1_predict = m1_reg_y1.predict(data_resp[:,:2]).reshape(-1,1)
  m1_noresp_y1_predict = m1_reg_y1.predict(data_noresp[:,:2]).reshape(-1,1)
  ## y2
  m1_reg_y2 = LinearRegression().fit(data_resp[:,:2], data_resp[:, 7])
  m1_resp_y2_predict = m1_reg_y2.predict(data_resp[:,:2]).reshape(-1,1)
  m1_noresp_y2_predict = m1_reg_y2.predict(data_noresp[:,:2]).reshape(-1,1)
  ## y3
  m1_reg_y3 = LinearRegression().fit(data_resp[:,:6], data_resp[:, 8])
  m1_resp_y3_predict = m1_reg_y3.predict(data_resp[:,:6]).reshape(-1,1)
  m1_noresp_y3_predict = m1_reg_y3.predict(data_noresp[:,:6]).reshape(-1,1)

  ## cktree imputation
  sim1_results_ckdtree[r, 0] = kdtree_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
  sim1_results_ckdtree[r, 1] = kdtree_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7])
  sim1_results_ckdtree[r, 2] = kdtree_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8])
  
  ## faiss imputation
  sim1_results_faiss[r, 0] = faiss_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
  sim1_results_faiss[r, 1] = faiss_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7])
  sim1_results_faiss[r, 2] = faiss_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8])

  ## faiss imputation using voronoi
  sim1_results_faiss_v[r, 0] = faiss_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6],voronoi = True) 
  sim1_results_faiss_v[r, 1] = faiss_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7],voronoi = True)
  sim1_results_faiss_v[r, 2] = faiss_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8],voronoi = True)

  ## annoy imputation
  sim1_results_annoy[r, 0] = annoy_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
  sim1_results_annoy[r, 1] = annoy_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7])
  sim1_results_annoy[r, 2] = annoy_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8])

  ## n2 imputation
  sim1_results_n2[r, 0] = n2_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
  sim1_results_n2[r, 1] = n2_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7])
  sim1_results_n2[r, 2] = n2_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8])

  ## flann imputation
  sim1_results_flann[r, 0] = flann_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
  sim1_results_flann[r, 1] = flann_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7])
  sim1_results_flann[r, 2] = flann_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8])

  ## scann imputation
  sim1_results_scann[r, 0] = scann_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
  sim1_results_scann[r, 1] = scann_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7])
  sim1_results_scann[r, 2] = scann_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8])

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [ ]:
[abs(np.mean(sim1_results_ckdtree, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0),
 abs(np.mean(sim1_results_faiss, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0),
 abs(np.mean(sim1_results_faiss_v, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0),
 abs(np.mean(sim1_results_annoy, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0),
 abs(np.mean(sim1_results_n2, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0),
 abs(np.mean(sim1_results_flann, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0),
 abs(np.mean(sim1_results_scann, axis = 0) - np.mean(data[:,[6,7,8]], axis = 0)) / np.mean(data[:,[6,7,8]], axis = 0)]

Save results to `dict` object and write to `pkl` file.

In [ ]:
results = {
    "data": pd.DataFrame(data),
    "sim1_results_ckdtree" : pd.DataFrame(sim1_results_ckdtree),
    "sim1_results_faiss" : pd.DataFrame(sim1_results_faiss),
    "sim1_results_faiss_v" : pd.DataFrame(sim1_results_faiss_v),
    "sim1_results_annoy": pd.DataFrame(sim1_results_annoy),
    "sim1_results_n2": pd.DataFrame(sim1_results_n2),
    "sim1_results_flann" : pd.DataFrame(sim1_results_flann),
    "sim1_results_scann": pd.DataFrame(sim1_results_scann)
          }

f = open("sim1-results.pkl","wb")
pickle.dump(results,f)
f.close()


## Processing results

In [ ]:
results = pd.read_pickle("sim1-results.pkl")
results.keys()

dict_keys(['data', 'sim1_results_ckdtree', 'sim1_results_faiss', 'sim1_results_annoy', 'sim1_results_n2', 'sim1_results_flann', 'sim1_results_faiss_v', 'sim1_results_scann'])

Calculate bias and standard error

In [ ]:
ys_true = np.array(np.mean(results["data"].loc[:, [6, 7, 8]], axis = 0))
ys_true

array([ 0.00049869, -0.00018652,  0.01020216], dtype=float32)

In [ ]:
algos = ["sim1_results_ckdtree", "sim1_results_faiss", "sim1_results_faiss_v",
         "sim1_results_annoy", "sim1_results_n2", "sim1_results_flann", 
         "sim1_results_scann"]
bias = np.zeros(shape = (len(algos), 3))
vars = np.zeros(shape = (len(algos), 3))

for i, a in enumerate(algos):
  bias[i,:] = np.array(np.mean(results[a], axis = 0)) - ys_true
  vars[i,:] = np.array(np.var(results[a], axis = 0))

ses = np.sqrt(vars)

algos_names = [str(i).replace("sim1_results_", "") for i in algos]

tab_report = np.column_stack((bias[:, 0], ses[:, 0], bias[:, 1], ses[:, 1], bias[:, 2], ses[:, 2]))

print(pd.DataFrame(np.round(tab_report*1000,4), 
                    index = algos_names, 
                    columns = ["bias_y1", "se_y1", "bias_y2", "se_y2", "bias_y3", "se_y3"]).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  bias\_y1 &    se\_y1 &  bias\_y2 &    se\_y2 &  bias\_y3 &    se\_y3 \\
\midrule
ckdtree &  -0.1764 &   3.4041 &  -0.1932 &   3.9165 &   0.0090 &   3.6757 \\
faiss   &  -0.5274 &   3.4219 &  -0.5769 &   3.9145 &  -0.0839 &   3.6274 \\
faiss\_v &  -0.1765 &   3.4657 &  -0.2439 &   3.9018 &   0.0638 &   3.6788 \\
annoy   &  -0.1747 &   3.4108 &  -0.1824 &   3.9220 &   0.0078 &   3.6805 \\
n2      &  -0.1730 &   3.4020 &  -0.1867 &   3.9200 &   0.0101 &   3.6781 \\
flann   &  -0.1771 &   3.4258 &  -0.1876 &   3.9400 &   0.0280 &   3.6748 \\
scann   &  -4.4243 &  14.2124 &  -3.3353 &  14.5755 &  -5.6344 &  11.4536 \\
\bottomrule
\end{tabular}



# Timings and costs

Timings for N = 50 000 (previously generated)

In [ ]:
%timeit -n 10 kdtree_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6])  
%timeit -n 10 faiss_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
%timeit -n 10 faiss_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6], voronoi = True) 
%timeit -n 10 faiss_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6], gpu = False)
%timeit -n 10 annoy_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
%timeit -n 10 n2_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
%timeit -n 10 flann_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
%timeit -n 10 scann_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 

10 loops, best of 3: 43.5 ms per loop
10 loops, best of 3: 169 ms per loop
10 loops, best of 3: 210 ms per loop
10 loops, best of 3: 1.24 s per loop
10 loops, best of 3: 4.86 s per loop
10 loops, best of 3: 2.74 s per loop
10 loops, best of 3: 1.08 s per loop
10 loops, best of 3: 664 ms per loop


Timings for N = 1 000 000

In [5]:
np.random.seed(123)
N = 1000000
x1 = np.random.uniform(size = N)
x2 = np.random.uniform(size = N)
x3 = np.random.uniform(size = N)
x4 = np.random.normal(size = N)
x5 = np.random.normal(size = N)
x6 = np.random.normal(size = N)
epsilon = np.random.normal(size=N)
y1 = -1 + x1 + x2 + epsilon
y2 = -1.167 + x1 + x2 + (x1 - 0.5)**2 + + (x2 - 0.5)**2 + epsilon
y3 = -1.5 + x1 + x2 + x3 + x4 + x5 + x6 + epsilon
p1 = np.exp(0.2 + x1 + x2) / (1 + np.exp(0.2 + x1 + x2))
data = np.column_stack((x1,x2,x3,x4,x5,x6,y1,y2,y3, p1)).astype('float32')
response_flag = np.random.binomial(n=1, p = p1, size = N)
data_resp = data[response_flag == 1]
data_noresp = data[response_flag != 1]
m1_reg_y1 = LinearRegression().fit(data_resp[:,:2], data_resp[:, 6])
m1_resp_y1_predict = m1_reg_y1.predict(data_resp[:,:2]).reshape(-1,1)
m1_noresp_y1_predict = m1_reg_y1.predict(data_noresp[:,:2]).reshape(-1,1)

In [ ]:
%timeit -n 10 kdtree_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6])  
%timeit -n 10 faiss_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 
%timeit -n 10 scann_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6]) 

10 loops, best of 3: 3.38 s per loop
10 loops, best of 3: 10.1 s per loop
10 loops, best of 3: 13.9 s per loop


# Approximate nearest neigbours using KD-TREE

In [79]:
np.random.seed(123)
N = 250000 ## 50000, 100000, 250000
x1 = np.random.uniform(size = N)
x2 = np.random.uniform(size = N)
x3 = np.random.uniform(size = N)
x4 = np.random.normal(size = N)
x5 = np.random.normal(size = N)
x6 = np.random.normal(size = N)
epsilon = np.random.normal(size=N)
y1 = -1 + x1 + x2 + epsilon
y2 = -1.167 + x1 + x2 + (x1 - 0.5)**2 + + (x2 - 0.5)**2 + epsilon
y3 = -1.5 + x1 + x2 + x3 + x4 + x5 + x6 + epsilon
p1 = np.exp(0.2 + x1 + x2) / (1 + np.exp(0.2 + x1 + x2))
data = np.column_stack((x1,x2,x3,x4,x5,x6,y1,y2,y3, p1)).astype('float32')

R = 500
sim_ann_naive = np.zeros(shape = (R, 3))
sim_ann_kdtree_y1 = np.zeros(shape = (R, 6))
sim_ann_kdtree_y2 = np.zeros(shape = (R, 6))
sim_ann_kdtree_y3 = np.zeros(shape = (R, 6))

for r in range(R):
  np.random.seed(r)
  print(r)
  response_flag = np.random.binomial(n=1, p = p1, size = N)
  data_resp = data[response_flag == 1]
  data_noresp = data[response_flag != 1]
  ## y1
  m1_reg_y1 = LinearRegression().fit(data_resp[:,:2], data_resp[:, 6])
  m1_resp_y1_predict = m1_reg_y1.predict(data_resp[:,:2]).reshape(-1,1)
  m1_noresp_y1_predict = m1_reg_y1.predict(data_noresp[:,:2]).reshape(-1,1)
  ## y2
  m1_reg_y2 = LinearRegression().fit(data_resp[:,:2], data_resp[:, 7])
  m1_resp_y2_predict = m1_reg_y2.predict(data_resp[:,:2]).reshape(-1,1)
  m1_noresp_y2_predict = m1_reg_y2.predict(data_noresp[:,:2]).reshape(-1,1)
  ## y3
  m1_reg_y3 = LinearRegression().fit(data_resp[:,:6], data_resp[:, 8])
  m1_resp_y3_predict = m1_reg_y3.predict(data_resp[:,:6]).reshape(-1,1)
  m1_noresp_y3_predict = m1_reg_y3.predict(data_noresp[:,:6]).reshape(-1,1)
  
  sim_ann_naive[r, :] = np.array([np.mean(data_resp[:, 6]), np.mean(data_resp[:, 7]), np.mean(data_resp[:, 8])])
  sim_ann_kdtree_y1[r, :] = np.array([kdtree_impute(m1_resp_y1_predict, m1_noresp_y1_predict, data_resp[:, 6], eps = i) for i in [0, 1, 5, 10, 50, 100]])
  sim_ann_kdtree_y2[r, :] = np.array([kdtree_impute(m1_resp_y2_predict, m1_noresp_y2_predict, data_resp[:, 7], eps = i) for i in [0, 1, 5, 10, 50, 100]])
  sim_ann_kdtree_y3[r, :] = np.array([kdtree_impute(m1_resp_y3_predict, m1_noresp_y3_predict, data_resp[:, 8], eps = i) for i in [0, 1, 5, 10, 50, 100]])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [80]:
[abs((np.mean(sim_ann_kdtree_y1, axis = 0) - np.mean(data[:, 6])) / np.mean(data[:, 6]))*100,
 abs((np.mean(sim_ann_kdtree_y2, axis = 0) - np.mean(data[:, 7])) / np.mean(data[:, 7]))*100,
 abs((np.mean(sim_ann_kdtree_y3, axis = 0) - np.mean(data[:, 8])) / np.mean(data[:, 8]))*100]

[array([7.78921202, 7.75129422, 7.90475606, 7.9292983 , 8.06087278,
        8.07808074]),
 array([7.27477802, 7.20647339, 6.92175057, 6.89905654, 6.83946352,
        6.82239987]),
 array([3.78411658, 3.8832197 , 4.12398733, 4.2377823 , 4.26537453,
        4.27871993])]

In [81]:
[np.sqrt((np.mean(sim_ann_kdtree_y1, axis = 0) - np.mean(data[:, 6]))**2 + np.var(sim_ann_kdtree_y1, axis = 0)),
 np.sqrt((np.mean(sim_ann_kdtree_y2, axis = 0) - np.mean(data[:, 7]))**2 + np.var(sim_ann_kdtree_y2, axis = 0)),
 np.sqrt((np.mean(sim_ann_kdtree_y3, axis = 0) - np.mean(data[:, 8]))**2 + np.var(sim_ann_kdtree_y3, axis = 0))]

[array([0.00159192, 0.00159084, 0.00159163, 0.001592  , 0.00159128,
        0.00159161]),
 array([0.00211822, 0.00211887, 0.00212029, 0.00211938, 0.00212058,
        0.00212044]),
 array([0.00155861, 0.00155882, 0.00155797, 0.00155644, 0.00155436,
        0.00155453])]

In [82]:
results = {
    "data": pd.DataFrame(data),
    "ann_y1" : pd.DataFrame(sim_ann_kdtree_y1),
    "ann_y2" : pd.DataFrame(sim_ann_kdtree_y2),
    "ann_y3" : pd.DataFrame(sim_ann_kdtree_y3),
    "naive" : pd.DataFrame(sim_ann_naive)
          }

f = open("sim-ann-kdtree-results-250000.pkl","wb")
pickle.dump(results,f)
f.close()